In [1]:
import time
import json
import random
import zipfile
import logging
import openpyxl
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
workbook = openpyxl.load_workbook('./doc/Input 2nd 10k.xlsx')
sheet = workbook.active

column_A = [cell.value for cell in sheet['A'][1:]]
_ids = [entry.split(';', 1)[0].replace(";", " ").replace('"','').strip() for entry in column_A]
names = [entry.split(';', 1)[1].split(";")[0].replace(";", " ").replace('"','').replace("  ", " ") for entry in column_A]
print(len(_ids), len(names))

9999 9999


In [3]:
def driver_return():
    user_agent = "Mozilla/5.0 (Linux; Android 9; SM-N975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.101 Mobile Safari/537.36"
    options = webdriver.ChromeOptions()
    options.add_argument("--window-size=150,500")
    options.add_argument("--disable-automation")
    options.add_argument("--no-sandbox")
    options.add_argument('--profile-directory=Default')
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--lang=en")
    options.add_argument("--enable-javascript")
    options.add_argument("--blink-settings=imagesEnabled=false")
    options.add_argument("--enable-cookies")
    options.add_experimental_option("detach", True)
    options.add_argument(f'--user-agent={user_agent}')
    driver = webdriver.Chrome(options=options)
    return driver

driver = driver_return()
driver.maximize_window()

In [4]:
def get_company_info(driver):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@role="main"]')))
    main_div = driver.find_element(By.XPATH, '//div[@role="main"]')
    if main_div:
        company = ""
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@role="main"]//a[contains(@data-tooltip, "Open website")]')))
            company_element = main_div.find_element(By.XPATH, '//a[contains(@data-tooltip, "Open website")]')
            if company_element:
                company = company_element.get_attribute('href')
                print("COMPANY_URL: ", company)
        except:
            print("COMPANY_URL NOT FOUND!")
            pass   
        
        service = ""
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@role="main"]//div[contains(@class, "lMbq3e")]//div[@class="fontBodyMedium"]')))
            service_element = main_div.find_element(By.XPATH, '//div[contains(@class, "lMbq3e")]//div[@class="fontBodyMedium"]')
            if service_element:
                service = service_element.text
                print("SERVICE_VAR: ", service)
        except:
            print("SERVICE_VAR NOT FOUND!")
            pass   
        return company, service
    return "", ""

def get_reviews(driver):
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@class="hjmQqc"]//div[@class="QGH3wd Inlyae"]')))
        counter = 0
        break_bool = False
        scroll_counter = 0
        prev_scroll_height = 0
        while True:
            all_reviews = driver.find_elements(By.XPATH, '//div[@class="hjmQqc"]//div[@class="QGH3wd Inlyae"]')
            if len(all_reviews) > counter:
                counter += 1
                
            total_scroll_counter = 0 
            for review in all_reviews[counter-1:counter]:
                if total_scroll_counter <= 10:
                    total_scroll_counter += 1
                    WebDriverWait(driver, 7).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="hjmQqc"]//div[@class="QGH3wd Inlyae"]')))
                    driver.execute_script("arguments[0].scrollIntoView(true);", review)        
                    time.sleep(0.3)
                    current_scroll_height = driver.execute_script("return Math.max( document.evaluate(\"//div[@class='nkePVe']\", document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.scrollHeight );")
                    if current_scroll_height == prev_scroll_height:
                        scroll_counter += 1
                    else:
                        scroll_counter = 0
                        
                    if scroll_counter > 9 and current_scroll_height == prev_scroll_height:
                        break_bool = True
                        break

                    prev_scroll_height = current_scroll_height
                else:
                    break_bool = True
                    break
            if break_bool:
                break
        
        visible_reviews = driver.find_elements(By.XPATH, '//div[@class="hjmQqc"]')
        reviews = []
        if visible_reviews:
            for index, review in enumerate(visible_reviews):
                item = {}
                html_content = review.get_attribute('outerHTML')
                soup = BeautifulSoup(html_content, 'html.parser')
                item['Counter'] = index + 1
                name_element = soup.find('div', class_='IaK8zc CVo7Bb')
                name = name_element.text
                item['Customer'] = name
                rating_element = soup.find('div', class_='nSAWhf').find('div')
                rating = rating_element['aria-label'].split("Rating of ")[1]
                item['Stars'] = rating
                time_element = soup.find('div', class_='bHyEBc')
                time_t = time_element.text
                item['Time'] = time_t
                description_element = soup.find('div', class_='nSAWhf').find('span')
                description = description_element.text
                item['Review'] = description
                reviews.append(item)
                if index >= 49:
                    break
        return reviews
    
    except Exception as e:
        print("Reviews Exception: ", e)
        return []

cnt = 0
already_crawled = set()
company_url = ""
service_var = ""

try:
    with open('googleSerp_v2.json', 'r', encoding='utf-8') as f:
        googleSerp = json.load(f)
        if googleSerp:
            for item in googleSerp:
                for key, value in item.items():
                    already_crawled.add(key)

    print(len(already_crawled))
except:
    print("NO PREVIOUS RECORD FOUND!")
    pass

for _id, name in zip(_ids, names):
    cnt += 1
    if _id not in already_crawled:
        print(cnt)
        try:
            driver.get('https://www.google.com/maps')
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="NtcBjb R30LOe eD6Rpc GnJVlc"]')))
            search_bar = driver.find_element(By.XPATH, '//div[@class="NtcBjb R30LOe eD6Rpc GnJVlc"]')
            search_bar.click()

            time.sleep(1)
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//input[@aria-labelledby="searchbox-placeholder"]')))
            search_bar_input = driver.find_element(By.XPATH, '//input[@aria-labelledby="searchbox-placeholder"]')
            for char in name:
                search_bar_input.send_keys(char)
            
            try:    
                WebDriverWait(driver, 8).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="BIg4wb"][1]/div/div[1]')))
                click_result = driver.find_element(By.XPATH, '//div[@class="BIg4wb"][1]/div/div[1]')
                if click_result:
                    click_result.click()
            except Exception as e:
                print(f"Exception occurred for {_id}: {e}")
                item = {}
                item[_id] = {
                    'company_info': False,
                    'company_name': name,
                    'address': "",
                    'phone_number': "",
                    'avg_customer_reviews': "",
                    'no_of_customer_reviews': "",
                    'service': "",
                    'company_url': ""
                }

                try:
                    address_element = driver.find_element(By.XPATH, '//button[contains(@aria-label, "Address")]')
                    item[_id]['address'] = address_element.text if address_element.text else ""
                except:
                    pass

                try:
                    phone_number_element = driver.find_element(By.XPATH, '//a[contains(@data-tooltip, "Call phone number")]')
                    item[_id]['phone_number'] = phone_number_element.text if phone_number_element.text else ""
                except:
                    pass

                try:
                    avg_reviews_element = driver.find_element(By.XPATH, '//div[@jsaction="pane.reviewChart.moreReviews"]//div[@class="fontDisplayLarge"]')
                    item[_id]['avg_customer_reviews'] = avg_reviews_element.text if avg_reviews_element.text else ""
                except:
                    pass

                try:
                    no_of_reviews_element = driver.find_element(By.XPATH, '//div[@jsaction="pane.rating.moreReviews"]//span[contains(@aria-label, "reviews")]')
                    item[_id]['no_of_customer_reviews'] = (no_of_reviews_element.text).split(" ")[0] if (no_of_reviews_element.text).split(" ")[0] else ""
                except:
                    pass
                item[_id]['reviews'] = []
                
                if _id not in already_crawled:
                    already_crawled.add(_id)
                    print(item, "\n")
                    with open('tempGoogleSerp_v2.json', 'a', encoding='utf-8') as f:
                        json.dump(item, f, ensure_ascii=False, indent=2)
                        f.write(',\n')
                else:
                    print("ALREADY_CRAWLED: ", _id)
                continue
            
            # Waiting for locations details to show
            WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="m6QErb DxyBCb kA9KIf dS8AEf "]')))
            company_url, service_var = get_company_info(driver)
            
            item = {}
            item[_id] = {
                'company_info': True,
                'company_name': "",
                'address': "",
                'phone_number': "",
                'avg_customer_reviews': "",
                'no_of_customer_reviews': "",
                'service': service_var,
                'company_url': company_url
            }
            
            try:
                company_name_element = driver.find_element(By.XPATH, '//h1[@class="DUwDvf fontTitleLarge"]')
                item[_id]['company_name'] = company_name_element.text if company_name_element.text else ""
            except:
                pass

            try:
                address_element = driver.find_element(By.XPATH, '//button[contains(@aria-label, "Address")]')
                item[_id]['address'] = address_element.text if address_element.text else ""
            except:
                pass

            try:
                phone_number_element = driver.find_element(By.XPATH, '//a[contains(@data-tooltip, "Call phone number")]')
                item[_id]['phone_number'] = phone_number_element.text if phone_number_element.text else ""
            except:
                pass

            try:
                avg_reviews_element = driver.find_element(By.XPATH, '//div[@jsaction="pane.reviewChart.moreReviews"]//div[@class="fontDisplayLarge"]')
                item[_id]['avg_customer_reviews'] = avg_reviews_element.text if avg_reviews_element.text else ""
            except:
                pass

            try:
                no_of_reviews_element = driver.find_element(By.XPATH, '//div[@jsaction="pane.rating.moreReviews"]//span[contains(@aria-label, "reviews")]')
                item[_id]['no_of_customer_reviews'] = (no_of_reviews_element.text).split(" ")[0] if (no_of_reviews_element.text).split(" ")[0] else ""
            except:
                pass
            
            try:
                WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.XPATH, '//div[@class="lMbq3e"]//span[contains(@aria-label, "review") or contains(@aria-label, "reviews")]')))
                reviews_click = driver.find_element(By.XPATH, '//div[@class="lMbq3e"]//span[contains(@aria-label, "review") or contains(@aria-label, "reviews")]')
                if reviews_click:
                    reviews_click.click()       
                    time.sleep(2)
                    item[_id]['reviews'] = get_reviews(driver)
            except Exception as e:
                print(f"Exception occurred for {_id}: {e}")
                item[_id]['reviews'] = []
            
            if _id not in already_crawled:
                already_crawled.add(_id)
                print(item, "\n")
                with open('tempGoogleSerp_v2.json', 'a', encoding='utf-8') as f:
                    json.dump(item, f, ensure_ascii=False, indent=2)
                    f.write(',\n')
            else:
                print("ALREADY_CRAWLED: ", _id)
            pass
            
        except Exception as e:
            print(f"Exception occurred for {_id}: {e}")
            item = {}
            item[_id] = {
                'company_info': False,
                'company_name': name,
                'address': "",
                'phone_number': "",
                'avg_customer_reviews': "",
                'no_of_customer_reviews': "",
                'service': "",
                'company_url': ""
            }
            
            try:
                company_name_element = driver.find_element(By.XPATH, '//h1[@class="DUwDvf fontTitleLarge"]')
                item[_id]['company_name'] = company_name_element.text if company_name_element.text else ""
            except:
                pass

            try:
                address_element = driver.find_element(By.XPATH, '//button[contains(@aria-label, "Address")]')
                item[_id]['address'] = address_element.text if address_element.text else ""
            except:
                pass

            try:
                phone_number_element = driver.find_element(By.XPATH, '//a[contains(@data-tooltip, "Call phone number")]')
                item[_id]['phone_number'] = phone_number_element.text if phone_number_element.text else ""
            except:
                pass

            try:
                avg_reviews_element = driver.find_element(By.XPATH, '//div[@jsaction="pane.reviewChart.moreReviews"]//div[@class="fontDisplayLarge"]')
                item[_id]['avg_customer_reviews'] = avg_reviews_element.text if avg_reviews_element.text else ""
            except:
                pass

            try:
                no_of_reviews_element = driver.find_element(By.XPATH, '//div[@jsaction="pane.rating.moreReviews"]//span[contains(@aria-label, "reviews")]')
                item[_id]['no_of_customer_reviews'] = (no_of_reviews_element.text).split(" ")[0] if (no_of_reviews_element.text).split(" ")[0] else ""
            except:
                pass
            item[_id]['reviews'] = []
            
            if _id not in already_crawled:
                already_crawled.add(_id)
                print(item, "\n")
                with open('tempGoogleSerp_v2.json', 'a', encoding='utf-8') as f:
                    json.dump(item, f, ensure_ascii=False, indent=2)
                    f.write(',\n')
            else:
                print("ALREADY_CRAWLED: ", _id)
            continue
    else:
        print("ALREADY_CRAWLED: ", _id)

110
ALREADY_CRAWLED:  5057466891
ALREADY_CRAWLED:  4612165432
ALREADY_CRAWLED:  4344100142
ALREADY_CRAWLED:  2852280449
ALREADY_CRAWLED:  9594365680
ALREADY_CRAWLED:  3146088685
ALREADY_CRAWLED:  9016192992
ALREADY_CRAWLED:  5049816963
ALREADY_CRAWLED:  3044678778
ALREADY_CRAWLED:  4989457522
ALREADY_CRAWLED:  4661428450
ALREADY_CRAWLED:  8485949337
ALREADY_CRAWLED:  3521909078
ALREADY_CRAWLED:  3354771512
ALREADY_CRAWLED:  5971785822
ALREADY_CRAWLED:  1804854329
ALREADY_CRAWLED:  5387023287
ALREADY_CRAWLED:  4639166377
ALREADY_CRAWLED:  6057892184
ALREADY_CRAWLED:  4980681622
ALREADY_CRAWLED:  4163820717
ALREADY_CRAWLED:  9750849519
ALREADY_CRAWLED:  4172133883
ALREADY_CRAWLED:  9257869713
ALREADY_CRAWLED:  9671066693
ALREADY_CRAWLED:  9084494398
ALREADY_CRAWLED:  1386052404
ALREADY_CRAWLED:  4760334079
ALREADY_CRAWLED:  3216783110
ALREADY_CRAWLED:  5743853025
ALREADY_CRAWLED:  9911270115
ALREADY_CRAWLED:  9530221428
ALREADY_CRAWLED:  3620819136
ALREADY_CRAWLED:  1801980605
ALREADY_CR